## Batch features from existing tables using Signals

This notebook creates a new feature view using the SDK that will be computed using stream processing.

### Flow of data

```mermaid
flowchart LR
    wh[(Warehouse)]
    cron[/Materialization CRON job/]
    signals(Signals)

    wh --> cron
    cron --> signals
```

---

### Define a new data source

Creates a data source with the source Snowflake table configuration.

In [ ]:
from snowplow_signals import BatchSource

data_source = BatchSource(
    name="ecommerce_transaction_interactions_source",
    database="SNOWPLOW_DEV1",
    schema="SIGNALS",
    table="SNOWPLOW_ECOMMERCE_TRANSACTION_INTERACTIONS_FEATURES",
    timestamp_field="UPDATED_AT",
)

### Create a feature view with the table fields

Feature views define the features inside the data sources and the entity types they relate to.

In [ ]:
from snowplow_signals import View, user_entity, Field

view = View(
    name="ecommerce_transaction_interactions_attributes",
    version=1,
    entity=user_entity,
    fields=[
        Field(
            name="TOTAL_TRANSACTIONS",
            type="int32",
        ),
        Field(
            name="TOTAL_REVENUE",
            type="int32",
        ),
        Field(
            name="AVG_TRANSACTION_REVENUE",
            type="int32",
        ),
    ],
    batch_source=data_source,
)

### Applying the data source and feature view to Signals

The following block pushes the data source and feature view definition to the Signals API and makes it available for a background CRON job that incrementally materializes the data from the warehouse table to the online feature store.

In [ ]:
from snowplow_signals import Signals

sp_signals = Signals(api_url="http://localhost:8000")
applied = sp_signals.apply([view])
print(f"{len(applied)} objects applied")

### Retrieving data

One can fetch the latest feature values for a particular user from the feature view as follow.

In [ ]:
response = sp_signals.get_online_attributes(
    view,
    identifiers=[
        "9999999999999999999999999",
    ],
)

response.to_dataframe()